In [1]:
# Imort main libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # for photing and viewing data
import matplotlib.pyplot as plt# plotting library
import plotly.express as px
import yfinance as yf
# import the TrendReq method from the pytrends request module
from pytrends.request import TrendReq
from sklearn.datasets import make_blobs
import datetime
from pathlib import Path
# Import VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
# import tensorflow and keras
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# Feature Selection using RFE
from sklearn.feature_selection import RFECV, RFE
from boruta import BorutaPy
# Import SHAP
import pandas_ta as ta
from sklearn.ensemble import RandomForestClassifier
import cufflinks as cf
cf.set_config_file(offline=True, theme='pearl')

import plotly.graph_objects as go
from talib import MA_Type
# Preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit, cross_val_score, KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit, TimeSeriesSplit
from sklearn.feature_selection import f_regression, SelectKBest, SelectPercentile
# scikit-learn modules
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import talib
# tensorflow modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM,RNN

from tensorflow.keras.optimizers import Adam, RMSprop 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import tensorflow as tf
# plotting & outputs
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from pprint import pprint
# Classifier
from sklearn.svm import LinearSVC, SVC

# Metrics
from sklearn.metrics import (
    precision_recall_curve, roc_curve, RocCurveDisplay, ConfusionMatrixDisplay,
    accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, auc,
    classification_report, confusion_matrix
)



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/zn/484pjr2d447dw2s8fwtrhw440000gn/T/ipykernel_24250/3212743309.py:50: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



In [2]:
df_price = yf.download(tickers = 'LULU', start="2001-05-21", end="2023-05-21", interval='1d')
df_price.head(5)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-07-27,12.50,14.320000,12.460,14.000,14.000,29257200
2007-07-30,14.43,15.925000,14.180,14.990,14.990,5767200
2007-07-31,16.00,17.084999,15.105,16.070,16.070,4441600
2007-08-01,16.49,16.625000,15.200,15.540,15.540,3063600
2007-08-02,16.00,16.915001,15.990,16.525,16.525,1969400


In [ ]:

df_price.ta.strategy('All',timed=True)

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

/opt/anaconda3/envs/tensorflow/lib/python3.9/multiprocessing/resource_tracker.py:96: UserWarning:

resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the loc

In [ ]:
def calculate_technical_indicators(df):
    
    #calculate the basic columne-price factors
    df['O-C'] = df.Open - df_price.Close
    df['H-L'] = df.High - df_price.Low
    df['Log_Return'] = np.log(df['Adj Close']).diff().dropna()
    
    df['Momentum_2'] = talib.MOM(df['Adj Close'], timeperiod=2)
    df['Momentum_3'] = talib.MOM(df['Adj Close'], timeperiod=3)
    df['Momentum_10'] = talib.MOM(df['Adj Close'], timeperiod=10)
    
    
    df['EMA_2'] = talib.EMA(df['Adj Close'], timeperiod=2)
    df['EMA_2'] = talib.EMA(df['Adj Close'], timeperiod=3)
    df['EMA_10'] = talib.EMA(df['Adj Close'], timeperiod=10)
    
    df['Lag_Return_2'] = df['Adj Close'].pct_change(periods=2)
    df['Lag_Return_3'] = df['Adj Close'].pct_change(periods=3)
    df['Lag_Return_10'] = df['Adj Close'].pct_change(periods=10)
    
    df['SMA_2'] = talib.SMA(df['Adj Close'], timeperiod=2)
    df['SMA_3'] = talib.SMA(df['Adj Close'], timeperiod=3)
    df['SMA_10'] = talib.SMA(df['Adj Close'], timeperiod=10)
    

    # Calculate Relative Strength Index (RSI)
    df['RSI'] = talib.RSI(df['Adj Close'], timeperiod=14)  # 14-period Relative Strength Index

    # Calculate Moving Average Convergence Divergence (MACD)
    macd, macdsignal, macdhist = talib.MACD(df['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_signal'] = macdsignal
    df['MACD_hist'] = macdhist

    # Calculate Bollinger Bands (BBANDS)
    upper, middle, lower = talib.BBANDS(df['Adj Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
    df['BBANDS_upper'] = upper
    df['BBANDS_middle'] = middle
    df['BBANDS_lower'] = lower

    # Calculate Average True Range (ATR)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Adj Close'], timeperiod=14)  # 14-period Average True Range

    # Calculate Stochastic Oscillator
    slowk, slowd = talib.STOCH(df['High'], df['Low'], df['Adj Close'], fastk_period=14, slowk_period=3, slowd_period=3)
    df['Stoch_slowk'] = slowk
    df['Stoch_slowd'] = slowd

    # Calculate Average Directional Movement Index (ADX)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Adj Close'], timeperiod=14)  # 14-period Average Directional Movement Index

    # Calculate Chaikin Oscillator
    df['Chaikin'] = talib.ADOSC(df['High'], df['Low'], df['Close'], df['Volume'], fastperiod=3, slowperiod=10)  # 3/10-period Chaikin Oscillator

    # Calculate On Balance Volume (OBV)
    df['OBV'] = talib.OBV(df['Close'], df['Volume'])

    # Calculate Average Directional Index (ADI)
    df['ADI'] = talib.AD(df['High'], df['Low'], df['Close'], df['Volume'])

    # Calculate Commodity Channel Index (CCI)
    df['CCI'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=20)  # 20-period Commodity Channel Index

    # Calculate Rate of Change (ROC)
    df['ROC'] = talib.ROC(df['Close'], timeperiod=10)  # 10-period Rate of Change

    # Calculate Money Flow Index (MFI)
    df['MFI'] = talib.MFI(df['High'], df['Low'], df['Close'], df['Volume'], timeperiod=14)  # 14-period Money Flow Index

    # Calculate Williams %R (WILLR)
    df['WILLR'] = talib.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14)  # 14-period Williams %R


    # Calculate Parabolic SAR (SAR)
    df['SAR'] = talib.SAR(df['High'], df['Low'], acceleration=0.02, maximum=0.2)

    # Calculate Trix (TRIX)
    df['TRIX'] = talib.TRIX(df['Close'], timeperiod=14)  # 14-period Trix

    # Calculate Ultimate Oscillator (ULTOSC)
    df['ULTOSC'] = talib.ULTOSC(df['High'], df['Low'], df['Close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)  # Ultimate Oscillator with periods 7, 14, and 28

    # Calculate Aroon Oscillator (AROONOSC)
    df['AROONOSC'] = talib.AROONOSC(df['High'], df['Low'], timeperiod=14)  # 14-period Aroon Oscillator
        # Add more technical indicators as needed

    return df

In [ ]:
df_price = df_price.astype('float64')

In [ ]:
indicators = calculate_technical_indicators(df_price)
df_price=pd.DataFrame(indicators)
df_price=df_price.reset_index()
df_price

In [ ]:
import pandas_datareader.data as web

# 设置起始和结束日期
start_date = '2001-01-01'
end_date = '2023-07-15'

# 使用pandas_datareader获取Fama-French五因子模型数据
ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start_date, end_date)
mom_data = web.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start_date, end_date)

# 提取因子数据
ff_factors = ff_data[0]
mom_factor = mom_data[0]
ff_factors=pd.concat([ff_factors, mom_factor], axis=1)
ff_factors=ff_factors.reset_index()

In [ ]:
ff_factors

In [ ]:
import eikon as ek
ek.set_app_key('e6749e154a70460f9886340bcda525e945f42b8e')

In [ ]:

df, err = ek.get_data(['LULU.O'], 
                      ['TR.Revenue.date','TR.Revenue','TR.GrossProfit'],
                      {'Scale': 6, 'SDate': 0, 'EDate': -100, 'FRQ': 'FQ'})
df['Date']=pd.to_datetime(df['Date']).dt.year
df=df.drop_duplicates()
df

In [ ]:
df1, err = ek.get_data(['LULU.O'], 
                      ["TR.TRESGScore.Date","TR.TRESGScore","TR.EnvironmentPillarScore",'TR.SocialPillarScore','TR.GovernancePillarScore'],
                      {'SDate':'0', 'EDate':'-19','Period':'FY0','Frq':'FY'})
df1['Date']=pd.to_datetime(df1['Date']).dt.year
df1=df1.dropna()
df = df.merge(df1.drop('Instrument', axis=1), on='Date')
df

In [ ]:
df['Date']=pd.to_datetime(df['Date'], format='%Y')
df=df.set_index('Date').resample('D').bfill().reset_index()

In [ ]:
upm=pd.read_excel('/Users/voona/Desktop/Conference/paper3/US_Policy_Uncertainty_Data.xlsx')
upm=upm.drop(upm.index[-1])
upm['Year']=upm['Year'].astype(int)
upm['Month']=upm['Month'].astype(int)
upm=upm[upm['Year']>2010]
upm['Date'] = upm['Year'].astype(str) + '-' + upm['Month'].astype(str)
upm=upm.drop(['Year','Month'],axis=1)
upm['Date']=pd.to_datetime(upm['Date'],format="%Y-%m")
upm=upm.set_index('Date').resample('D').bfill().reset_index()

In [ ]:
df=df.merge(upm,on='Date')

## Machine Learning part

In [156]:
# execute the TrendReq method by passing the host language (hl) and timezone (tz) parameters
pytrends = TrendReq(hl='en-US', tz=360)
# build list of keywords
kw_list = ["lululemon"] 

# build the payload
pytrends.build_payload(kw_list, timeframe='2011-01-01 2022-01-01', geo='US')
df_google = pytrends.interest_over_time().reset_index()
df_google=df_google.rename(columns={'date':"Date",'lululemon':"GoogleIndex"})
df_google=df_google.set_index('Date').resample('D').bfill().reset_index()
df_google=df_google.drop('isPartial', axis=1)

In [157]:
df_google

,Date,GoogleIndex
0,2011-01-01,13
1,2011-01-02,12
2,2011-01-03,12
3,2011-01-04,12
4,2011-01-05,12
...,...,...
4014,2021-12-28,81
4015,2021-12-29,81
4016,2021-12-30,81
4017,2021-12-31,81


In [158]:
df=df.merge(df_google,on='Date')
df=df.merge(df_price,on='Date')

In [159]:
df=df.merge(ff_factors,on='Date')

In [160]:
df_Features= df.drop(['Open', 'High', 'Low', 'Close', 'Instrument',], axis=1)

In [162]:
df_Features.columns

Index(['Date', 'Revenue', 'Gross Profit', 'ESG Score',
       'Environmental Pillar Score', 'Social Pillar Score',
       'Governance Pillar Score', 'Three_Component_Index',
       'News_Based_Policy_Uncert_Index', 'GoogleIndex', 'Adj Close', 'Volume',
       'O-C', 'H-L', 'Log_Return', 'Momentum_2', 'Momentum_3', 'Momentum_10',
       'EMA_2', 'EMA_10', 'Lag_Return_2', 'Lag_Return_3', 'Lag_Return_10',
       'SMA_2', 'SMA_3', 'SMA_10', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist',
       'BBANDS_upper', 'BBANDS_middle', 'BBANDS_lower', 'ATR', 'Stoch_slowk',
       'Stoch_slowd', 'ADX', 'Chaikin', 'OBV', 'ADI', 'CCI', 'ROC', 'MFI',
       'WILLR', 'SAR', 'TRIX', 'ULTOSC', 'AROONOSC', 'Pass_Returns_2',
       'Pass_Returns_3', 'Pass_Returns_5', 'Momentum_5', 'SMA_5', 'EMA',
       'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '],
      dtype='object')

In [75]:
df_Features.to_csv('/Users/voona/Desktop/考证/CQF exam/data.csv')

In [81]:
#set the threshold for the extreme near zero value 
# To aviod the loss the potentional gains , we have to set the thrwshold as lower as possible.
df_Features['Sign'] = np.where(df_Features['Log_Return']>0,1,0)
df_Features['Sign'] = df_Features['Sign'].shift(-1)
df_Features=df_Features.dropna()
X = df_Features.loc[:,[
       'News_Based_Policy_Uncert_Index', 'GoogleIndex', 'Adj Close', 'Volume',
       'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'BBANDS_upper',
       'BBANDS_middle', 'BBANDS_lower', 'ATR', 'Stoch_slowk', 'Stoch_slowd',
       'ADX', 'Chaikin', 'OBV', 'ADI', 'CCI', 'ROC', 'MFI', 'WILLR', 'SAR',
       'TRIX', 'ULTOSC', 'AROONOSC', 'O-C', 'H-L', 'Log_Return',
       'Pass_Returns_2', 'Pass_Returns_3', 'Pass_Returns_5', 'Momentum_2',
       'Momentum_3', 'Momentum_5', 'SMA_2', 'SMA_3', 'SMA_5', 'EMA', 'Mkt-RF',
       'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   ']]
feature_names = X.columns
y = df_Features['Sign']
y = y.values
# pandas-ta converts all dtype to objects
y = y.astype(int) 
df_Features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2765 entries, 0 to 2764
Data columns (total 55 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Date                            2765 non-null   datetime64[ns]
 1   Revenue                         2765 non-null   Float64       
 2   Gross Profit                    2765 non-null   Float64       
 3   ESG Score                       2765 non-null   Float64       
 4   Environmental Pillar Score      2765 non-null   Float64       
 5   Social Pillar Score             2765 non-null   Float64       
 6   Governance Pillar Score         2765 non-null   Float64       
 7   Three_Component_Index           2765 non-null   float64       
 8   News_Based_Policy_Uncert_Index  2765 non-null   float64       
 9   GoogleIndex                     2765 non-null   int64         
 10  Adj Close                       2765 non-null   float64       
 11  Volume   

## Missing Values

In [55]:

# check for missing values
df_Features.isna().sum()

Date                              0
Revenue                           0
Gross Profit                      0
ESG Score                         0
Environmental Pillar Score        0
Social Pillar Score               0
Governance Pillar Score           0
Three_Component_Index             0
News_Based_Policy_Uncert_Index    0
GoogleIndex                       0
Adj Close                         0
Volume                            0
RSI                               0
MACD                              0
MACD_signal                       0
MACD_hist                         0
BBANDS_upper                      0
BBANDS_middle                     0
BBANDS_lower                      0
ATR                               0
Stoch_slowk                       0
Stoch_slowd                       0
ADX                               0
Chaikin                           0
OBV                               0
ADI                               0
CCI                               0
ROC                         

## EDA

## Feature Selection

In [76]:
# Feature scaling
scaler = MinMaxScaler()
# For each X, calculate VIF and save in dataframe

def vif(X):
# perform feature scaling
    xs = scaler.fit_transform(X)
    # subsume into a dataframe
    vif = pd.DataFrame()
    vif["Features"] = X.columns
    vif["VIF Factor"] = [variance_inflation_factor(xs, i) for i in range(xs.shape[1])]
    return vif

In [77]:
# List scores
vif_score=vif(X).round(2)
#Drop VIF score > 5
X_method1=X.loc[:,vif_score[vif_score["VIF Factor"]<6]['Features']]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning:

divide by zero encountered in scalar divide



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [58]:
# SelectKBest
method2 = SelectKBest(f_regression, k=8)
# selector1 = SelectPercentile(f_regression, percentile=25)
# Fit the model
method2.fit(X,y)
# Show selected features
method2.get_support(indices=True)

array([ 1,  7, 17, 29, 39, 41, 44, 45])

In [59]:
# Iterate the score
m2_feature=pd.DataFrame({'feature':X.columns,'F-score':method2.scores_}).sort_values('F-score')
m2_feature.tail(8)

,feature,F-score
1,GoogleIndex,0.809973
44,RF,0.860144
41,HML,0.917835
29,Pass_Returns_2,1.044601
7,MACD_hist,1.121281
17,ADI,2.094412
45,Mom,2.633107
39,Mkt-RF,4.043138


In [60]:
# Filter six features with highest score
X_method2 = X.loc[:,m2_feature.tail(8)['feature']]
X_method2

,GoogleIndex,RF,HML,Pass_Returns_2,MACD_hist,ADI,Mom,Mkt-RF
0,12,0.0,0.77,0.033872,-0.351827,3.946467e+06,-0.01,1.18
1,12,0.0,0.07,-0.046762,-0.472709,-1.868466e+06,-0.58,-0.26
2,12,0.0,0.13,-0.026136,-0.503842,1.139930e+06,0.12,0.59
3,12,0.0,-0.34,0.040998,-0.564967,-1.416376e+05,-0.51,-0.15
4,12,0.0,-0.27,-0.018865,-0.592217,3.096398e+05,0.35,-0.21
...,...,...,...,...,...,...,...,...
2761,81,0.0,0.19,0.049177,-5.785360,1.287580e+08,0.58,2.06
2762,81,0.0,-0.36,0.051513,-4.406893,1.292290e+08,0.95,0.97
2763,81,0.0,-0.48,-0.026702,-3.013866,1.299415e+08,0.09,0.71
2764,81,0.0,0.29,0.009004,-1.268159,1.306256e+08,1.93,1.22


Class Imbalance
A major problem in classification that can arise depending on the data set available is class imbalance. This means, in the context of binary labels, that the frequency of one particular class compared to the other class might be higher. This might lead to situations in which the [learning algorithm] neural network simply predicts the class with the higher frequency since this already can lead to low loss and high accuracy values. By applying appropriate weights, one can make sure that both classes gain equal importance during the [ML] DNN training step

In [61]:
# class frequency
c =  df_Features['Sign'].value_counts()
c


Sign
1.0    1421
0.0    1345
Name: count, dtype: int64

In [62]:
df_Features.columns

Index(['Date', 'Revenue', 'Gross Profit', 'ESG Score',
       'Environmental Pillar Score', 'Social Pillar Score',
       'Governance Pillar Score', 'Three_Component_Index',
       'News_Based_Policy_Uncert_Index', 'GoogleIndex', 'Adj Close', 'Volume',
       'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'BBANDS_upper',
       'BBANDS_middle', 'BBANDS_lower', 'ATR', 'Stoch_slowk', 'Stoch_slowd',
       'ADX', 'Chaikin', 'OBV', 'ADI', 'CCI', 'ROC', 'MFI', 'WILLR', 'SAR',
       'TRIX', 'ULTOSC', 'AROONOSC', 'O-C', 'H-L', 'Log_Return',
       'Pass_Returns_2', 'Pass_Returns_3', 'Pass_Returns_5', 'Momentum_2',
       'Momentum_3', 'Momentum_5', 'SMA_2', 'SMA_3', 'SMA_5', 'EMA', 'Mkt-RF',
       'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   ', 'Sign'],
      dtype='object')

In [63]:
data=df_Features.drop('Date',axis=1)

In [64]:
# class weight function
def cwts(dfs):
    c0, c1 = np.bincount(dfs['Sign'])
    w0=(1/c0)*(len(df))/2 
    w1=(1/c1)*(len(df))/2 
    return {0: w0, 1: w1}
# check class weights
class_weight = cwts(data)
class_weight


{0: 1.029368029739777, 1: 0.974313863476425}

In [65]:
# With the calculated weights, both classes gain equal weight
class_weight[0] * c[0], class_weight[1] * c[1]

(1384.5, 1384.5)

## Split Data

In [87]:

# Always keep shuffle = False for financial time series
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# convert to array
X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test) 

# Output the train and test data size
print(f"Train and Test Size {len(X_train)}, {len(X_test)}")

Train and Test Size 2212, 553


In [73]:
y_train

array([0, 1, 0, ..., 1, 0, 1])

In [67]:
# define random forest classifier
forest = RandomForestClassifier(n_jobs=-1, 
                                class_weight=cwts(data), 
                                random_state=42, 
                                max_depth=5)

# train the model
forest.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 1.029368029739777,
                                     1: 0.974313863476425},
                       max_depth=5, n_jobs=-1, random_state=42)

In [68]:
# print scores
print("Accuracy Score \t\t", accuracy_score(y_test, forest.predict(X_test)))

Accuracy Score 		 0.4747292418772563


## Borutapy
Bortua algorithm is designed to take the “all-relevant” approach to feature selection. It is a wrapper built around the random forest classification algorithm which is relatively quick and can run without tuning of parameters and it gives a numerical estimate of the feature importance.

In [78]:

# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=0)

# find all relevant features
# takes input in array format not as dataframe
feat_selector.fit(X_train, y_train) 

# check selected features
print(feat_selector.support_)

# check ranking of features
print(feat_selector.ranking_)

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X_train)


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In stock forecasting, the datasets of different stock markets are diverse based on the perspective of machine learning models. The datasets broadly consist of two sets of data, namely intrinsic data and extrinsic data. Intrinsic data is primarily information mined from the stock data itself, including historical stock prices, financial indices and other technical analysis data. Due to the intrinsic properties of stock data, most of its intrinsic data are time series. External data can be highly variable and includes textual information, underlying data, industry knowledge graphs, etc.

1. Stock prices: Stock prices most directly reflect the performance of the stock market and are both an input feature to the model and a predictive target. Stock prices are commonly used in the papers included in this research. However, based on different model designs, stock prices contain several types, including OPEN, HIGH, LOW and CLOSE data.

2. Technical Analysis Indicators: Technical analysis indicators are commonly used tools in conventional stock analysis. These indicators are highly correlated with stock market performance and include practical financial indicators such as exchange rates, P/E ratios and volume.

3. Macroeconomic data: Macroeconomic information reflects financial conditions. The Consumer Price Index (CPI) and Gross Domestic Product (GDP) are two commonly used indices that correlate with the stock market on which they are based, as they indicate market conditions and the nature of stock market rallies or falls.

4. Underlying data: Underlying data describes the details of an economic entity and may include financial position, corporate structure and any other information published to shareholders. However, only a small proportion of the underlying data is utilised in deep learning models, with low reporting frequency and unstructured textual information being the two main limitations.

5. Knowledge graphs: Companies in different industries may be intrinsically linked, for example, two companies merging into one, and supply chains may be impacted. In some recent experiments, knowledge graphs from open source can now be integrated with traditional stock data and further improve the performance of the model.

Textual information: Textual information covers text from a variety of sources, including mainly news and reports as well as social media posts and user comments. As most textual information is unstructured, sentiment analysis is a common method of exploitation through deep machine learning. Information can be classified into several categories (positive, neutral or negative) for further use.

## LSTM

In [31]:
'''
features= df_Features.loc[:,[
       'News_Based_Policy_Uncert_Index', 'GoogleIndex', 'Adj Close', 'Volume',
       'O-C', 'H-L', 'Log_Return',  'Pass_Returns_2', 'Pass_Returns_3',
       'Pass_Returns_5', 'Momentum_2', 'Momentum_3', 'Momentum_5', 'SMA_2',
       'SMA_3', 'SMA_5', 'EMA']].values
'''
target= df_Features['Sign'].values
features=df_Features.loc[:,[  'News_Based_Policy_Uncert_Index', 'GoogleIndex', 'Adj Close', 'Volume',
       'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'BBANDS_upper',
       'BBANDS_middle', 'BBANDS_lower', 'ATR', 'Stoch_slowk', 'Stoch_slowd',
       'ADX', 'Chaikin', 'OBV', 'ADI', 'CCI', 'ROC', 'MFI', 'WILLR', 'SAR',
       'TRIX', 'ULTOSC', 'AROONOSC', 'O-C', 'H-L', 'Log_Return',
       'Pass_Returns_2', 'Pass_Returns_3', 'Pass_Returns_5', 'Momentum_2',
       'Momentum_3', 'Momentum_5', 'SMA_2', 'SMA_3', 'SMA_5', 'EMA', 'Mkt-RF',
       'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   ']]

In [32]:
features=X_method2
X_method2

,Environmental Pillar Score,RF,HML,MACD_hist,Pass_Returns_2,ADI,Mom,Mkt-RF
0,5.959752,0.0,0.77,-0.351827,0.033872,3.946467e+06,-0.01,1.18
1,5.959752,0.0,0.07,-0.472709,-0.046762,-1.868466e+06,-0.58,-0.26
2,5.959752,0.0,0.13,-0.503842,-0.026136,1.139930e+06,0.12,0.59
3,5.959752,0.0,-0.34,-0.564967,0.040998,-1.416376e+05,-0.51,-0.15
4,5.959752,0.0,-0.27,-0.592217,-0.018865,3.096398e+05,0.35,-0.21
...,...,...,...,...,...,...,...,...
2763,69.54404,0.0,-0.48,-3.013866,-0.026702,1.299415e+08,0.09,0.71
2764,69.54404,0.0,0.29,-1.268159,0.009004,1.306256e+08,1.93,1.22
2765,69.54404,0.0,0.81,-0.019902,-0.008798,1.296943e+08,-0.53,-0.27
2766,69.54404,0.0,0.17,0.953918,-0.020877,1.294467e+08,0.41,0.06


In [33]:
# Normalize the feature data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# Create rolling windows of size window_size
window_size = 30
X = []
y = []
for i in range(window_size, len(scaled_features)):
    X.append(scaled_features[i-window_size:i])
    y.append(target[i])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))




Epoch 1/10
69/69 [==============================] - 3s 23ms/step - loss: 0.6950 - accuracy: 0.5196
Epoch 2/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6927 - accuracy: 0.5233
Epoch 3/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6954 - accuracy: 0.5178
Epoch 4/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6933 - accuracy: 0.5018
Epoch 5/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6925 - accuracy: 0.5201
Epoch 6/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6928 - accuracy: 0.5100
Epoch 7/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6928 - accuracy: 0.5169
Epoch 8/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6923 - accuracy: 0.5228
Epoch 9/10
69/69 [==============================] - 2s 23ms/step - loss: 0.6921 - accuracy: 0.5233
Epoch 10/10
18/18 [==============================] - 0s 9ms/step - loss: 0.6945 - accuracy: 0.5055
Accuracy: 

In [ ]:

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model on the testing set
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

In [89]:
# perform normalization
scaler = MinMaxScaler()
scaledtrain = scaler.fit_transform(X_train)
scaledtest = scaler.transform(X_test)

In [122]:
from time_series_generator import TimeseriesGenerator

# sequence length
seqlen = 21

# number of features
numfeat = X.shape[1]
# generate train and test sequence data
g = TimeseriesGenerator(scaledtrain, y_train, length=seqlen)
g_ = TimeseriesGenerator(scaledtest, y_test, length=seqlen)
# verify length
len(g), len(g_)




(1, 1)

In [123]:
# check feature set
g[0][0]

array([[[1.84714830e-01, 0.00000000e+00, 1.35844992e-02, ...,
         4.36532508e-01, 0.00000000e+00, 5.13440860e-01],
        [1.84714830e-01, 0.00000000e+00, 7.33503502e-04, ...,
         3.96284830e-01, 0.00000000e+00, 4.36827957e-01],
        [1.84714830e-01, 0.00000000e+00, 4.51838426e-03, ...,
         4.02476780e-01, 0.00000000e+00, 5.30913978e-01],
        ...,
        [1.84714830e-01, 0.00000000e+00, 4.87047400e-03, ...,
         2.66253870e-01, 0.00000000e+00, 4.91935484e-01],
        [1.84714830e-01, 0.00000000e+00, 5.07585408e-03, ...,
         4.08668731e-01, 0.00000000e+00, 5.95430108e-01],
        [1.84714830e-01, 0.00000000e+00, 1.83375876e-02, ...,
         4.39628483e-01, 9.09090909e-02, 5.83333333e-01]],

       [[1.84714830e-01, 0.00000000e+00, 7.33503502e-04, ...,
         3.96284830e-01, 0.00000000e+00, 4.36827957e-01],
        [1.84714830e-01, 0.00000000e+00, 4.51838426e-03, ...,
         4.02476780e-01, 0.00000000e+00, 5.30913978e-01],
        [1.84714830e-01, 

In [125]:
# check target 
g[0][1]


array([1, 1, 1, ..., 1, 0, 1])

In [126]:
# verify batch size
for i in range(len(g)):
    a, b = g[i]
    print(a.shape, b.shape)

(2191, 21, 46) (2191,)


In [107]:
# Create a sequential model
def create_model(hu=256, lookback=60, features=1):
    
    tf.keras.backend.clear_session()   

    # instantiate the model
    model = Sequential()
    
    model.add(LSTM(units=hu*2, input_shape=(lookback, features), activation = 'elu', return_sequences=True, name='LSTM1'))
    model.add(Dropout(0.4, name='Drouput1'))
    
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=True, name='LSTM2'))
    model.add(Dropout(0.4, name='Drouput2'))
    
    model.add(LSTM(units=hu, activation = 'elu', return_sequences=False, name='LSTM3'))
    
    model.add(Dense(units=1, activation='sigmoid', name='Output'))             
    
    # specify optimizer separately (preferred method))
    opt = Adam(lr=0.001, epsilon=1e-08, decay=0.0)       
    
    # model compilation - 'binary_crossentropy' - 'accuracy' - BinaryAccuracy(name='accuracy', threshold=0.5)
    model.compile(optimizer=opt, 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=['accuracy'])

    return model

In [108]:
tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [109]:
# lstm network
model = create_model(hu=10, lookback=seqlen, features=numfeat)

In [110]:
# summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LSTM1 (LSTM)                (None, 21, 20)            5360      
                                                                 
 Drouput1 (Dropout)          (None, 21, 20)            0         
                                                                 
 LSTM2 (LSTM)                (None, 21, 10)            1240      
                                                                 
 Drouput2 (Dropout)          (None, 21, 10)            0         
                                                                 
 LSTM3 (LSTM)                (None, 10)                840       
                                                                 
 Output (Dense)              (None, 1)                 11        
                                                                 
Total params: 7,451
Trainable params: 7,451
Non-trainabl

In [114]:
# plot layers
plot_model(model, to_file='./img/model.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Train the Model

In [116]:
results_path = Path('results', 'lstm_time_series')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [121]:
g

In [127]:
import datetime as dt
# Specify callback functions
model_path = (results_path / 'model.h5').as_posix()
logdir = os.path.join("./tensorboard/logs", dt.datetime.now().strftime("%Y%m%d-%H%M%S"))

my_callbacks = [
    EarlyStopping(patience=20, monitor='loss', mode='min', verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath=model_path, verbose=1, monitor='loss', save_best_only=True),
    TensorBoard(log_dir=logdir, histogram_freq=1)
]


In [131]:
# Model fitting
model.fit(X_train,
          epochs=500, 
          verbose=1, 
          callbacks=my_callbacks, 
          shuffle=False,
          class_weight=class_weight)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# plot layers
plot_model(model, to_file='./img/model.png', show_shapes=True, show_layer_names=True)
Train the Model
# Specify callback functions
model_path = (results_path / 'model.h5').as_posix()
logdir = os.path.join("./tensorboard/logs", dt.datetime.now().strftime("%Y%m%d-%H%M%S"))

my_callbacks = [
    EarlyStopping(patience=20, monitor='loss', mode='min', verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath=model_path, verbose=1, monitor='loss', save_best_only=True),
    TensorBoard(log_dir=logdir, histogram_freq=1)
]
# Model fitting
model.fit(g,
          epochs=500, 
          verbose=1, 
          callbacks=my_callbacks, 
          shuffle=False,
          class_weight=class_weight)